## Constant Volatility Fee Calculations

Reproducing the constant volatility invariant to determine feeTier examples from [Guillaume's Constant Volatility AMM blog post](https://lambert-guillaume.medium.com/designing-a-constant-volatility-amm-e167278b5d61#:~:text=TL%3BDR%3A%20Constant%20volatility%20AMMs,dynamics%20of%20the%20underlying%20assets.)

Equation below:

# ![Uniswap v3 fee tier formula](https://miro.medium.com/v2/resize:fit:1400/format:webp/1*yah6u1x9uQkC7llZrz9QhQ.png)


In [15]:
import math

In [184]:
# fee = iv_per_year / (2 * math.sqrt(365 * 24 * 60 * 60)) * math.sqrt(tick_tvl / amount0) * math.sqrt(deltaT_secs)

def getFeeTierDecimal(iv_per_year, tick_tvl, amount0, deltaT_secs):
  two_sqrt_t = (2 * math.sqrt(365 * 24 * 60 * 60))
  # print('two_sqrt_t: ', two_sqrt_t) # > 11231.384598525687

  iv_seconds = iv_per_year / two_sqrt_t
  # print('iv_seconds: {:.10f}'.format(iv_seconds))

  liq_swap_ratio = tick_tvl / amount0
  # print('liq_swap_ratio', liq_swap_ratio)

  sqrt_liq_swap_ratio = math.sqrt(liq_swap_ratio)
  # print('sqrt_liq_swap_ratio_t', sqrt_liq_swap_ratio)

  sqrt_delta_t = math.sqrt(deltaT_secs)
  # print('sqrt_delta_t', sqrt_delta_t)

  # print("final calc. iv_seconds | sqrt_liq_swap_ratio | sqrt_delta_t ", iv_seconds, sqrt_liq_swap_ratio, sqrt_delta_t)
  feeTier = iv_seconds * sqrt_liq_swap_ratio * sqrt_delta_t

  return feeTier


# return fee in ten thousandths (1 100th of a basis point)
# also modified to replicate fixed point math ops in sol
def getFee(iv_per_year, tick_tvl, amount0, deltaT_secs):
  two_sqrt_t = 2 * math.sqrt(365 * 24 * 60 * 60)
  # print('two_sqrt_t: ', two_sqrt_t) # > 11231.384598525687

  iv_seconds =  iv_per_year / two_sqrt_t
  # print('iv_seconds: {:.10f}'.format(iv_seconds))

  liq_swap_ratio = tick_tvl / amount0
  print('liq_swap_ratio', liq_swap_ratio)

  sqrt_liq_swap_ratio = math.sqrt(liq_swap_ratio)
  # print('sqrt_liq_swap_ratio', sqrt_liq_swap_ratio)

  sqrt_delta_t = math.sqrt(deltaT_secs)
  # print('sqrt_delta_t', sqrt_delta_t)

  # print("final calc. iv_seconds | sqrt_liq_swap_ratio | sqrt_delta_t ", iv_seconds, sqrt_liq_swap_ratio, sqrt_delta_t)
  fee = iv_seconds * sqrt_liq_swap_ratio * sqrt_delta_t

  return fee

In [185]:
# Example 1: 1 ETH trade with 100% annual volatility target
IV_per_year = 1_000_000  # 100% annual volatility, in fee units (1 / 100ths of a bip). As a decimal, 1, times 10 ** 6 (fee tier decimals) = 1_000_000
FEE_DECIMALS = 1_000_000
tick_tvl = 315           # Example TVL at tick
amount0 = 1 # 1 ETH trade

deltaT_secs = 15   # 1 block (15 seconds)

fee1Pct = getFeeTierDecimal(IV_per_year/FEE_DECIMALS, tick_tvl, amount0, deltaT_secs)
print(f"Fee (decimal) for 1 ETH trade after 1 block: {fee1Pct:.4}")
print(f"Fee (pct) for 1 ETH trade after 1 block: {fee1Pct:.4%}")
fee1 = getFee(IV_per_year, tick_tvl, amount0, deltaT_secs)
print(f"Fee for 1 ETH trade after 1 block: {fee1}")
print(f"Fee (to %) for 1 ETH trade after 1 block: {fee1 // 10_000}%")
print('')
# > Fee for 1 ETH trade after 1 block: 0.6120%

# Example 2 - 2 blocks
deltaT_secs = 30   # 2 blocks
fee2Pct = getFeeTierDecimal(IV_per_year/FEE_DECIMALS, tick_tvl, amount0, deltaT_secs)
print(f"Fee for 1 ETH trade after 2 blocks: {fee2Pct:.4%}")
print(f"Fee (pct) for 1 ETH trade after 2 blocks: {fee2Pct:.4%}")
fee2 = getFee(IV_per_year, tick_tvl, amount0, deltaT_secs)
print(f"Fee for 1 ETH trade after 2 blocks: {fee2:.10}")
print(f"Fee (to %) for 1 ETH trade after 2 blocks: {fee2 / 10_000:.10}%")
print('')
# > Fee for 1 ETH trade after 2 blocks: 0.8655%

# Example 3: 1000 ETH whale trade
amount0 = 1000     # 1000 ETH trade
deltaT_secs = 15   # 1 block
fee3Pct = getFeeTierDecimal(IV_per_year/FEE_DECIMALS, tick_tvl, amount0, deltaT_secs)
print(f"Fee (decimal) for 1000 ETH trade after 1 block: {fee3Pct:.4}")
print(f"Fee (pct) for 1000 ETH trade after 1 block: {fee3Pct:.4%}")
fee3 = getFee(IV_per_year, tick_tvl, amount0, deltaT_secs)
print(f"Fee for 1000 ETH trade after 1 block: {fee3:.10}")
print(f"Fee (to %) for 1000 ETH trade after 1 block: {fee3 / 10_000:.10}%")
print('')
# Fee for 1000 ETH whale trade: 0.0194%

# Total fees collected by LPs from whale trade
total_fees = amount0 * fee3Pct
print(f"Total fees collected from whale trade: {total_fees:.2f} ETH")
# Total fees collected from whale trade: 0.19 ETH

Fee (decimal) for 1 ETH trade after 1 block: 0.00612
Fee (pct) for 1 ETH trade after 1 block: 0.6120%
liq_swap_ratio 315.0
Fee for 1 ETH trade after 1 block: 6120.228082418327
Fee (to %) for 1 ETH trade after 1 block: 0.0%

Fee for 1 ETH trade after 2 blocks: 0.8655%
Fee (pct) for 1 ETH trade after 2 blocks: 0.8655%
liq_swap_ratio 315.0
Fee for 1 ETH trade after 2 blocks: 8655.309559
Fee (to %) for 1 ETH trade after 2 blocks: 0.8655309559%

Fee (decimal) for 1000 ETH trade after 1 block: 0.0001935
Fee (pct) for 1000 ETH trade after 1 block: 0.0194%
liq_swap_ratio 0.315
Fee for 1000 ETH trade after 1 block: 193.5386054
Fee (to %) for 1000 ETH trade after 1 block: 0.01935386054%

Total fees collected from whale trade: 0.19 ETH


## Calculating IV in circuit
The ZK circuit is provided with amount1 and raw liquidity values.

Here we take the mock amount1 and raw liquidity values provided in unit tests and use them to calculate IV so we know what output value to expect from the circuit

In [9]:
amount1_0 = 1564800000000000000000
amount1_1 = 1469250000000000000000
totalVolume = amount1_0 + amount1_1 # 3034050000000000000000
currentTick = 200169
liq = 17525466147715557006

In [46]:
# Constants from FixedPoint96 library
RESOLUTION = 96
Q96 = 0x1000000000000000000000000  # 79228162514264337593543950336


def get_amount1_for_liquidity(
    sqrt_ratio_a_x96: int,
    sqrt_ratio_b_x96: int,
    liquidity: int
) -> int:
    if sqrt_ratio_a_x96 > sqrt_ratio_b_x96:
        sqrt_ratio_a_x96, sqrt_ratio_b_x96 = sqrt_ratio_b_x96, sqrt_ratio_a_x96

    # Equivalent to FullMath.mulDiv in Solidity
    return (liquidity * (sqrt_ratio_b_x96 - sqrt_ratio_a_x96)) // Q96

def get_amount1_for_liquidity_tick_space(
    tick_a: int,
    tick_b: int,
    liquidity: int
) -> int:
    if tick_a > tick_b:
        tick_a, tick_b = tick_b, tick_a

    return (liquidity * (tick_b - tick_a))

# Calculate tick to sqrt price
def tick_to_sqrt_price_x96(tick):
    return int(1.0001 ** (tick / 2) * 2**96)

# Calculate for both functions
sqrt_price_current = tick_to_sqrt_price_x96(currentTick)
sqrt_price_next = tick_to_sqrt_price_x96(currentTick + 1)

# Using first function
liq_amount1 = get_amount1_for_liquidity_tick_space(currentTick, currentTick+1, liq)
print(f"Amount1 using get_amount1_for_liquidity_1: {liq_amount1}")

# iv = 2 * (feeTier / 10 ** 6) * (dailyVolume / tickTvl) ** 0.5 * Math.sqrt(365)

# start with tickTvl term
# For calculating tickTvl in the circuit, approximate it by trating in range liquidity as one tick 
# d0 is derivedEth value which for weth=1. decs0=18 for weth
print("liq: ", liq)
print("currenttTick: ", currentTick)
tickTvl = liq_amount1
print(f"Tick TVL: {tickTvl}")

# iv
# Calculate IV (in percent) using the formula:
# iv = 2 * (feeTier / 10 ** 6) * (dailyVolume / tickTvl) ** 0.5 * Math.sqrt(365)

dailyVolume = totalVolume

# Fee tier is 500 bps (0.05%)
feeTier = 500

# Calculate IV, but don't divide by fee tier decimals to leave IV in fee tier units. this is how the target_iv is represented in the smart contract
iv = 2 * feeTier * math.sqrt(dailyVolume / tickTvl) * math.sqrt(365)

print(f"Daily Volume: {dailyVolume}")
print(f"IV: {iv}")

# For comparison with the circuit output (uint248 value)
iv_scaled = int(iv)
print(f"IV: {iv_scaled}")

# 1675084242

Amount1 using get_amount1_for_liquidity_1: 17525466147715557006
liq:  17525466147715557006
currenttTick:  200169
Tick TVL: 17525466147715557006
Daily Volume: 3034050000000000000000
IV: 251375.53611852165
IV: 251375


In [ ]:
19454766237951424304